In [12]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [2]:
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("training on GPU")
else:
    device = torch.device("cpu")
    print("training on CPU")

training on CPU


In [101]:
class CNN_RNN(nn.Module):
    
    def __init__(self):
        super(CNN_RNN, self).__init__()
        self.conv1d_1 = nn.Conv1d(1, 8, 10, stride = 10)
        self.relu_1 = nn.ReLU()
        self.conv1d_2 = nn.Conv1d(8, 16, 10,stride = 10)
        self.relu_2 = nn.ReLU()
        self.conv1d_3 = nn.Conv1d(16, 16, 10,stride = 10)
        self.relu_3 = nn.ReLU()
        
        self.gru_1 = nn.GRU(input_size = 150, hidden_size = 32,
                          num_layers = 2, dropout=0.2, batch_first=True)
        
        self.fc = nn.Linear(32, 1)
        self.relu_4 = nn.ReLU()
        
    def forward(self, x, h):
        out = self.conv1d_1(x)
        out = self.relu_1(out)
        out = self.conv1d_2(out)
        out = self.relu_2(out)
        out = self.conv1d_3(out)
        out = self.relu_3(out)
        
        out, h = self.gru_1(out, h)
        out = self.fc(out)
        out = self.relu_4(out)
        
        return out, h
    
    def init_hidden(self, batch_size):
        is_cuda = torch.cuda.is_available()
        if is_cuda:
            device = torch.device("cuda")
        else:
            device = torch.device("cpu")
        weight = next(self.parameters()).data
        hidden = weight.new(2, batch_size, 32).zero_().to(device)
        return hidden
        

In [8]:
%%time
raw_data = pd.read_csv('../data/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


In [9]:
X_train = raw_data.acoustic_data.values
y_train = raw_data.time_to_failure.values

In [10]:
#This cell is for getting complete sequence: which means cut when the time to failure is zero
ends_mask = np.less(y_train[:-1], y_train[1:])
segment_ends = np.nonzero(ends_mask)

train_segments = []
start = 0
for end in segment_ends[0]:
    train_segments.append((start, end))
    start = end
    
print(train_segments)

[(0, 5656573), (5656573, 50085877), (50085877, 104677355), (104677355, 138772452), (138772452, 187641819), (187641819, 218652629), (218652629, 245829584), (245829584, 307838916), (307838916, 338276286), (338276286, 375377847), (375377847, 419368879), (419368879, 461811622), (461811622, 495800224), (495800224, 528777114), (528777114, 585568143), (585568143, 621985672)]


In [94]:
class Data(Dataset):
    def __init__(self, x_df, y_df, segments, batch_size, ts_length, steps):
        self.x = x_df
        self.y = y_df
        self.segments = segments
        self.batch_size = batch_size
        self.ts_len = ts_length
        self.steps = steps
        self.segments_size = np.array([s[1] - s[0] for s in segments])
        self.segments_p = self.segments_size / self.segments_size.sum()
            
    def __len__(self):
        return self.steps
    
    def __getitem__(self, idx):
        segment_index = np.random.choice(range(len(self.segments)), p=self.segments_p)
        segment = self.segments[segment_index]
        end_indexes = np.random.randint(segment[0] + self.ts_len, segment[1], size=self.batch_size)

        x_batch = np.empty((self.batch_size, self.ts_len))
        y_batch = np.empty(self.batch_size, )

        for i, end in enumerate(end_indexes):
            x_batch[i, :] = self.x[end - self.ts_len: end]
            y_batch[i] = self.y[end - 1]
            
        #normalize
#         x_batch = (x_batch - self.x_mean)/self.x_std

        return torch.from_numpy(np.expand_dims(x_batch, axis=2)).view(64,1,150000),torch.from_numpy(y_batch)


In [95]:
data_loader = Data(X_train, y_train, train_segments, 64, 150000, 400)

In [103]:
def train_model(train_loader, lr = 0.01, hidden_dim = 256, epochs = 2):
    model = CNN_RNN()
    
    h = model.init_hidden(64)
    model.to(device)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    epoch_times = []

    print('=========> Starting training')
    for epoch in range(epochs):
        for i, (data, labels) in enumerate(train_loader):
            outputs, h = model(data.float(), h)
            loss = criterion(outputs, labels.float())
            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()
            if i % 10 == 0:
                print(f'[Epoch {epoch+1}/2, Step {i}/{100}]  loss: {loss.item(): .4f}')
    return model

In [104]:
model = train_model(data_loader)

=========> Starting training
[Epoch 1/2, Step 0/100]  loss:  86.2633


KeyboardInterrupt: 

In [105]:
#save the trained model
model_name = None  # change this to your desired model name
torch.save(model, './result/'+ model_name)

NameError: name 'model' is not defined

In [107]:
#load the model for testing
path = None  # change this to your path
model = torch.load(PATH)
model.eval()

NameError: name 'PATH' is not defined